### Causes of packet loss

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.ticker import FormatStrFormatter

In [2]:
def create_bins(lower_bound, width, quantity):
    """ create_bins returns an equal-width (distance) partitioning.
        It returns an ascending list of tuples, representing the intervals.
        A tuple bins[i], i.e. (bins[i][0], bins[i][1])  with i > 0
        and i < quantity, satisfies the following conditions:
            (1) bins[i][0] + width == bins[i][1]
            (2) bins[i-1][0] + width == bins[i][0] and
                bins[i-1][1] + width == bins[i][1]
    """
    bins = []
    for low in range(lower_bound, lower_bound + quantity * width + 1, width):
        bins.append((low, low + width))
    return bins

In [3]:
def bin_fields(df, fields, bin_width=10, bin_quantity=49):
    """
    Bins multiple dfs into a single dictionary that can be used as an average for multiple fields across multiple
    runs
    :param df: dataframe to bin
    :param fields: fields to be binned.
    :param bin_width: width of each bin
    :param bin_quantity: total number of bins
    :return:
    """
    bins = create_bins(lower_bound=0, width=bin_width, quantity=bin_quantity)
    distances = []
    overall_fields = {}
    for interval in bins:
        upper_b = interval[1]
        distances.append(upper_b)

    for field in fields:
        overall_fields[field] = []

    overall_fields["distance"] = distances

    distance_col = "txRxDistanceTB"

    for i in range(len(bins)):
        lower_b = bins[i][0]
        upper_b = bins[i][1]
        fields_temp = df[(df[distance_col] >= lower_b) & (df[distance_col] < upper_b)]
        for field in fields:
            if i < len(overall_fields[field]):
                overall_fields[field][i] = (fields_temp[field].mean() + overall_fields[field][i]) / 2
            else:
                overall_fields[field].append(fields_temp[field].mean())

    return overall_fields

In [4]:
def errors_dist(distances, decoded, decoded_labels, errors, error_labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    for i in range(len(decoded)):
        ax.plot(distances, decoded[i], label=decoded_labels[i], marker=self.markers[i], markevery=3)

        for j in range(len(errors[i])):
            ax.plot(distances, errors[i][j], label=error_labels[i][j], marker=self.markers[i + j])

    ax.legend(loc='center left')
    ax.set(xlabel='Distance (m)', ylabel='Messages Decoded')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=.1))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))
    
    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

In [5]:
interested_folder = [
    "After-5-high-density",
    "DCC-Enabled",
    "NO-CC",
    "After-1",
    "After-1-high-density",
    "NO-CC-high-density",
    "Highway-fast",
    "After-5",
    "Random-Access-Medium-Density",
    "Random-Access-Packet-Dropping-Medium-Density",
    "RRI-Adaptation-CR-limit-MD",
    "RRI-Adaptation-DCC-MD"
]

In [ ]:
raw_data_folder = "/Users/brianmccarthy/git_repos/results-analysis/data/parsed_data/cv2x"
loaded_dfs = {}

In [ ]:
for folder in os.listdir(raw_data_folder):
    if folder in interested_folder:
        print("="*50)
        print("Parsing configuration: {}".format(folder))
        print("="*50)
        loaded_dfs[folder] = read_data_in(os.path.join(raw_data_folder, folder))

In [ ]:
extra_folders = ["RRI-Adaptation-DCC-MD"]
for folder in os.listdir(raw_data_folder):
    if folder in extra_folders:
        print("="*50)
        print("Parsing configuration: {}".format(folder))
        print("="*50)
        loaded_dfs[folder] = read_data_in(os.path.join(raw_data_folder, folder))

In [ ]:
distances = create_bins(0, 25, 20)

In [ ]:
for i in range(len(distances)):
    distances[i] = distances[i][1]

In [ ]:
pdrs = {}
ipgs = {}
for configuration in interested_folder:
    df = loaded_dfs[configuration]
    
    # Filter the times down
#     df = df[df["Time"] > 502]
    
    # Put config back into our dictionary
#     loaded_dfs[configuration] = df
    
    # Calculate pdr
    pdr = []
    ipg = []
    pdr_binned = bin_fields(df, ["tbDecoded"])
    ipg_binned = bin_fields(df, ["interPacketDelay"])
    
    for i in range(len(pdr_binned["tbDecoded"])):
        if i < len(pdr):
            pdr[i] = (pdr_binned["tbDecoded"][i] + pdr[i]) / 2
        else:
            pdr.append(pdr_binned["tbDecoded"][i])
            
    for i in range(len(ipg_binned["interPacketDelay"])):
        if i < len(ipg):
            ipg[i] = (ipg_binned["interPacketDelay"][i] + ipg[i]) / 2
        else:
            ipg.append(ipg_binned["interPacketDelay"][i])
            
    for i in range(len(pdr)):
        pdr[i] = pdr[i] * 100
    pdrs[configuration] = pdr
    
    ipgs[configuration] = ipg